# Lab - Time Series Classification 
Deep Learning indenfor det sundhedstekniske domæne (Fall 2021)<br>
Instructor: Rahman Peimankar (abpe@mmmi.sdu.dk)

The goal of this lab is to revise the learning of using RNN/LSTM, CNN, and combined RNN+LSTM models in Lesson 8 and Lesson 9, which are focused on bio-signals classification.

# Dataset & Problem Definition

In the first part of the lab, we use an Electrocardiograms (ECG) datset freely available from ``PhyioNet`` website: https://physionet.org/.   

The dataset is called ``MIH-BIH Arrythmia dataset (MIT-BIH)``, which can be used to classify different heart diseases. We use this dataset to detect abnormal heartbeats from normal ones. You can read more about the dataset here: https://physionet.org/content/mitdb/1.0.0/ 

To start working with the MIT-BIH dataset, we need to learn how to use a Python library called ``WFDB``, which is developed for downloading reading, writing, and processing bio-signals and annotations/labels available on PhysioNet website. For more information, you may look into the ``WFDB`` GitHub repository here: https://github.com/MIT-LCP/wfdb-python

You may also check the ``demo.ipynb`` notebook to learn more about how to use ``WFDB``: https://github.com/MIT-LCP/wfdb-python/blob/master/demo.ipynb

# Part 1: Data Preparation

In this section, we prepare the data using ``WFDB`` package: 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import os

Let's download the entire dataset and save them in the folder named ``data`` on our computer!

**Note:** If you have not already installed ``WFDB``, you can do it as follows: 

In [2]:
!pip install wfdb

In [3]:
import wfdb
os.mkdir('./data')  # This makes a new folder/directory named 'data' for you.
os.chdir('./data')  # This changes the current directory to the newly created 'data'.
wfdb.dl_database('mitdb', os.getcwd())  # This will download the entire dataset into the newly created 'data' folder.

Generating record list for: 100
Generating record list for: 101
Generating record list for: 102
Generating record list for: 103
Generating record list for: 104
Generating record list for: 105
Generating record list for: 106
Generating record list for: 107
Generating record list for: 108
Generating record list for: 109
Generating record list for: 111
Generating record list for: 112
Generating record list for: 113
Generating record list for: 114
Generating record list for: 115
Generating record list for: 116
Generating record list for: 117
Generating record list for: 118
Generating record list for: 119
Generating record list for: 121
Generating record list for: 122
Generating record list for: 123
Generating record list for: 124
Generating record list for: 200
Generating record list for: 201
Generating record list for: 202
Generating record list for: 203
Generating record list for: 205
Generating record list for: 207
Generating record list for: 208
Generating record list for: 209
Generati

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

There are ECG records of 48 patients available in the MIT-BIH dataset. We need this list later for reading the annotation of each ECG record.

In [4]:
patients = ['100','101','102','103','104','105','106','107',
            '108','109','111','112','113','114','115','116',
            '117','118','119','121','122','123','124','200',
            '201','202','203','205','207','208','209','210',
            '212','213','214','215','217','219','220','221',
            '222','223','228','230','231','232','233','234']

We will import and use ``WFDB`` package to download the ECG signals/records and their corresponding annotations. The annotation files contain the assigned labels to each heartbeat saying whether it is normal or not. There are many different annotation symbols and we need to clean the data to consider the ones that we need to develop our model.

* ``wfdb.rdann`` command inside the below ``for`` loop will read the annotation file of each signal.

In [ ]:
import wfdb

df = pd.DataFrame()

for patient_id in patients:
    file_name = os.getcwd() + '\\' + patient_id
    annotation = wfdb.rdann(file_name, 'atr')
    types = annotation.symbol
    
    values, counts = np.unique(types, return_counts=True)
    df1 = pd.DataFrame({'types':values, 'val':counts, 'patient_id':[patient_id]*len(counts)})
    df = pd.concat([df, df1],axis = 0)

Now, we can print the number of heartbeats for each heartbeat type as shown below:

In [ ]:
df.groupby('types').val.sum().sort_values(ascending = False)

However, some of these symbols represent the "non-beat" beats and we need to filter them out!

For more information about the beat types, please see the links below: 

https://archive.physionet.org/physiobank/database/html/mitdbdir/tables.htm#testbeats


https://archive.physionet.org/physiobank/database/html/mitdbdir/intro.htm#symbols

In [ ]:
non_beat = ['[','!',']','x','(',')','p','t','u','`','\'','^','|','~','+','s','T','*','D','=','"','@','Q','?']

abnormal_beat = ['L','R','V','/','A','f','F','j','a','E','J','e','S']

Q1. Please complete the code below in order to group the three categories ``normal``, ``abnormal``, and ``nonbeat``. You should assign 0, 1, and -1 to ``normal``, ``abnormal``, and ``nonbeat``, respectively. By doing this, we can see the distribution of the classes/categories.

**Hint:** You should first create a new column named ``class`` and initilaize it with -1. Then, change the values to 0 if is is ``N`` (normal beat) and to 1 if it is ``abnormal``. You should get a similr result as below. 

<center>
<img src="fig1.JPG" width="300"/>

In [ ]:
df['class'] = -1

# YOUR CODE HERE
raise NotImplementedError()

Great! So far, we have downloaded the dataset and have read the annotation files corresponding to each ECG record. You have also converted the annotations/labels into three groups (i.e. Normal, Abnormal, and Non-beats).

Now it is the time to implement a function that can load the ECG signal and annotation file for a single patient. The ``read_ecg`` function gets a path to the downloaded dataset and returns the signal values/amplitudes, annotation/label type, and the location of the annotations/labels.

**Note:** As given in the MIT-BIH dataset website (https://physionet.org/content/mitdb/1.0.0/), the sampling frequency of the ECG records is 360 Hz.

**Note:** The annotations/lables of each heartbeat are usually placed at the peak of beat (or the QRS segment). Please see figure below. This will help us to cut/extract each beat fairly accurately. These heartbeats will be used as inputs for our deep learning models.



In [ ]:
def read_ecg(path):
    
    # Read ECG signal
    record = wfdb.rdrecord(path)
    
    # Read corresponding annotation file
    annot = wfdb.rdann(path, 'atr')
    
    # Get the ECG signal
    ecg_sig = record.p_signal
    
    # Sampling frequency should be 360 Hz
    # assert record.fs == 360, 'sample freq is not 360'
    
    # Get annotations' symbols/types and their corresponding location (index)
    annot_type = annot.symbol
    annot_sample = annot.sample
    
    return ecg_sig, annot_type, annot_sample

Let's see an example on how to use the function ``read_ecg``.

In [ ]:
path = os.getcwd() + '/' + patients[0]  # This reads the first ECG records, which is record number ´100´
ecg_sig, annot_type, annot_sample = read_ecg(path)

Let's see the number of beats for each type in the first ECG record ('100').

You can see that there are in total 11 **non-beats** shown as ``+`` and ``~`` and 53 **abnormal** beats shown as ``J`` and ``v``. Also, there are 2700 **normal** beats for the first record.

In [ ]:
symbol, count = np.unique(types, return_counts=True)
for symb, c in zip(symbol, count):
    print(symb, c)

Figure below shows an excerpt of the first ECG record that we read using ``read_ecg`` function. As you can see, there is one **abnormal** beat at index around 66765. 

In the next step, we develop a function to extract every beat by receiving the center index (e.g. 66765) of the beat and looking $\pm$ 3 seconds (or $2\times 3 seconds \times 360 Hz = 2060 samples$) around it. 

<center>
<img src="fig2.JPG" width="600"/> 

Q2. Please complete the ``build_dataset`` function below to make the dataset by ignoring the **non-beats** heartbeats. You will use the ``read_ecg`` function to read each ECG reacords and then extract the single beats as described in the previous cell.

**Note:** 

1. The ``build_dataset`` function gets the list of patients (``patients``), the time interval (``interval``) before and after each heart peak ($\pm$3 seconds for example), sampling frequency (``fs``), which is 360 Hz for MIT-BIH dataset, and the list of abnormal beats symbols (``abnormal_beat``) as defined earlier.



2. The ``build_dataset`` function returns two matrices called ``X`` and ``Y``, which are extracted heartbeats and their coresponding labels (**normal** or **abnormal**), respectively.  It also returns the annotation symbol (``annot_symb``) for each individual extracted beat. It should be mentioned that matrix X rows and columns represent the number of beats and the values of beats (i.e. ``interval`` * ``fs``).

In [ ]:
def  (patients, interval, fs, abnormal_beat):
    
    # Initialize the arrays
    num_cols = 2 * interval * fs  # This specify the length of the heartbeats to be extracted.
    X = np.zeros((1,num_cols))
    Y = np.zeros((1,1))
    annot_symb = []
    
    # This list stores the number of extracted heartbeats for each patient.
    num_beats = []
    
    for patient_id in patients:
        file_path = os.getcwd() + '\\' + patient_id
        
        ecg_sig, annot_type, annot_sample = read_ecg(file_path)
        
        # Since there are two leads of ECGs for each record, we only select the first lead to work with.
        ecg_sig = ecg_sig[:,0]
        
        # We simply remove the "non-beats" beats from the df_annot dataframe and only keep "normal" and "abnormal" beats.
        df_annot = pd.DataFrame({'annot_type':annot_type,
                              'annot_sample':annot_sample})
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # The "make_XY" builds the x and y matrics for each extarcted heartbeat
        x, y, symbol = make_XY(ecg_sig, df_annot, interval, num_cols, abnormal_beat)
        annot_symb = annot_symb + symbol
        num_beats.append(x.shape[0])
        X = np.append(X,x,axis = 0)
        Y = np.append(Y,y,axis = 0)

    return X, Y, annot_symb



def make_XY(ecg_sig, df_annot, interval, num_cols, abnormal_beat):
    # this function builds the X,Y matrices for each beat
    # it also returns the original symbols for Y
    
    num_row = len(df_annot)

    x = np.zeros((num_row, num_cols))
    y = np.zeros((num_row,1))
    symbol = []
    
    # count the rows
    row_size = 0

    for annot_sample, annot_type in zip(df_annot.annot_sample.values, df_annot.annot_type.values):

        left = max([0,(annot_sample - interval*fs) ])
        right = min([len(ecg_sig),(annot_sample + interval*fs) ])
        xx = ecg_sig[left: right]
        if len(xx) == num_cols:
            x[row_size,:] = xx
            y[row_size,:] = int(annot_type in abnormal_beat)
            symbol.append(annot_type)
            row_size += 1
    x = x[:row_size,:]
    y = y[:row_size,:]
    return x, y, symbol



interval =  3
fs = 360

X, Y, annot_symb = build_dataset(patients, interval, fs, abnormal_beat)

Congratulations! You have prepared a complete dataset for the deep learning model development. This is fantastic because you can use the same codes with very litle changes to work with any other databases from PhysioNet such as:

* MIT-BIH Supraventricular Arrhythmia Database (https://physionet.org/content/svdb/1.0.0/)
* St Petersburg INCART 12-lead Arrhythmia Database (https://physionet.org/content/incartdb/1.0.0/) 

# Part 2: Model Development

Let's start the fun part of every machine/deep learning project, model development :) 

 Q3. It is the time to split the prepared dataset into train and validation parts. Please complete the code below using ``train_test_split`` function from ``scikit-learn`` library to split the dataset. 
 
**Note:** Please set only these two parameters of ``train_test_split`` function and leave the rest as default:

``test_size=0.33`` and ``random_state=42``

And name the outputs of the function as: ``X_train``, ``X_validation``, ``y_train``, ``y_validation``

In [ ]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
raise NotImplementedError()

Before we start to develope a CNN and an LSTM model for classify heart arrhytmia, let's have a baseline dense neural networks (with only two dense layers) to compare the results of CNN and LSTM with it.

If you have not installed tensorflow already, follow one of the following to install it:

1) using simple "pip" install:

``!pip install tensorflow``

2) Open your "Anaconda Prompt" and type in the below command to install:

``conda install -c conda-forge tensorflow``

In [ ]:
import tensorflow 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(rate = 0.2))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size = 32, epochs= 10, verbose = 1)

Let's check how well our trained model perform on the validation data. 

In [ ]:
from sklearn.metrics import classification_report

y_pred_val = model.predict(X_validation)

threshold = 0.5
print(classification_report(y_validation, y_pred_val>threshold))

Q3. Now it is your turn to develop a CNN model. Please complete the cosde below to develope a model architecture as below figure:
    
<center>
<img src="fig3.JPG" width="500"/>  
    
 
**Note:** You should set the ``input_shape = (2160, 1)``. This is equal to the length of one heartbeat ($2\times 3 seconds \times 360 Hz = 2060 samples$) 
    
**Important:** It may take a while to train the below CNN model on CPU and that is why only 2 epochs is chosen. You may train it on Google Colab on GPU nodes.

Another important step to do here is to reshape the inputs to be ``[num_samples, interval, feature=1]``. You may notice that unlike image datasets that we had ``feature=3`` representing the three channels (RGB), here we have only 1D time series as inputs.

In [ ]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_valid_reshaped = np.reshape(X_validation, (X_validation.shape[0], X_validation.shape[1], 1))

print(X_train_reshaped.shape)
print(X_valid_reshaped.shape)

In [ ]:
from tensorflow.keras.layers import Conv1D

model = Sequential()
# YOUR CODE HERE
raise NotImplementedError()

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train_reshaped, y_train, batch_size = 32, epochs= 2, verbose = 1)

In [ ]:
model.summary()

Q4. Please complete the code below to report the performance of the CNN model.

**Note:** Please name the model predictions on the validation data (``X_valid_reshaped``) as: ``y_pred_cnn``!

In [ ]:
from sklearn.metrics import classification_report

# YOUR CODE HERE
raise NotImplementedError()

threshold = 0.5
print(classification_report(y_validation, y_pred_cnn>threshold))

We can also plot a Confusion Matrix for our trained classifier on the validation data! You should use ``confusion_matrix`` method/function from scikit-learn library for this purpose. For more help and see some examples on how to use``confusion_matrix``, please read here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html  

Q5. Please complete the code below for calculating the confusion matrix for the validation data and name it ``cm``.

**Hint:** The confusion matrix sould look like the below:

<center>
<img src="fig5.JPG" width="400"/> 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# YOUR CODE HERE
raise NotImplementedError()

cm = cm / cm.astype(np.float).sum(axis=1) # This line normalizes the calculated confusion matrix

figure = plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

Q6. Now it is time to develop an LSTM model. Please complete the cosde below to develope a model architecture as below figure:
    
<center>
<img src="fig4.JPG" width="500"/>  
    
 
**Note 1:** You should set the ``input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])``. These are equal to the number of samples (heartbeats) and length of one heartbeat ($2\times 3 seconds \times 360 Hz = 2060 samples$). 
    
**Note 2:** The ``return_sequences=True`` force the LSTM/BiLSTM layer to return the output for each LSTM cell/unit.    
    
**Important:** It may take a while to train the below LSTM model on CPU and that is why only 2 epochs is chosen. You may train it on Google Colab on GPU nodes. Here, we reduce the size of the dataset (``X_train_cnn[:10000], y_train[:10000]``) to make it feasible for quick prototyping but you should train the model with the full dataset.

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM

model_lstm = Sequential()
# YOUR CODE HERE
raise NotImplementedError()

model_lstm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_lstm.fit(X_train_reshaped[:10000], y_train[:10000], batch_size = 32, epochs= 1, verbose = 1)

In [ ]:
model_lstm.summary()

Q7. Please complete the code below to report the performance of the LSTM model.

**Note:** Please name the model predictions on the validation data (``X_valid_reshaped``) as: ``y_pred_lstm``!

In [ ]:
from sklearn.metrics import classification_report

# YOUR CODE HERE
raise NotImplementedError()

threshold = 0.5
print(classification_report(y_validation, y_pred_lstm>threshold))

# Next Steps

Well done! You have learnt how to develop CNN and LSTM models on real-world time series datasets. But, what can be done to improve our model?


* There are some parameters in the CNN and LSTM models that can be optimized such as:
        1. number of filters in the CNN model.
        2. number of layers in the LSTM model. 


* The model architectures such as number of layers can be explored further to improve the performance.


* We can develop a new model by combining CNN and LSTM layers.


* We can use a completely unseen dataset to test our trained model. For this purpose, two datasets can be used:
        1.MIT-BIH Supraventricular Arrhythmia Database (https://physionet.org/content/svdb/1.0.0/)
        2. St Petersburg INCART 12-lead Arrhythmia Database (https://physionet.org/content/incartdb/1.0.0/)  
        
        
##### So, if you are interested in learning more about time series classification using ECG signals and doing your final report (and/or perhaps your Master thesis) on this topic, please contact me :)  

# Thank you!